In [ ]:
%matplotlib inline
import cv2
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = (12, 9)

# Camera Calibration

In this notebook we will explore a quick example of camera calibration using a 3D object.

First, load the image.

In [ ]:
img_path = 'data/checkerboard.png'
img = plt.imread(img_path)[:, :, :3]

plt.imshow(img)
plt.show()

Calibration works by relating pixel coordinates on the image to real world coordinates in some world frame. In order to do this, we need to find points in the image of which we know the real world coordinates. Calibration objects are nice in that they have well defined features that can serve as these points, with measured coordinates for these features. Shown above is our calibration object. Each smaller square has dimensions 18mm x 18mm.

(2 points) Convert the image to grayscale, and use OpenCV's `cv2.cornerHarris()` to find the corners of the pattern in the calibration object. `cv2.cornerHarris()` returns a heatmap of "cornerness" scores. Pick a threshold on the scores. Display the thresholded scores by making a copy of the heatmap, setting all pixels with strong response to 1.0 and others to 0.0.

Now that we have found the corners of the object, we need to know the pixel coordinates and the coordinates in the real world. Below are pixel coordinates corresponding to six of the corners of the calibration object. We can mark their locations in the image.

In [ ]:
corners = [
    (648, 789),
    (729, 690),
    (519, 753),
    (1044, 770),
    (976, 899),
    (908, 445),
]

plt.imshow(img)
for x, y in corners:
    plt.plot(x, y, color=(0, 1, 0), marker='o', markersize=10)
plt.show()

Now we have the pixel coordinates, we need the corresponding real world coordinates for those corners. Below are the real world coordinates for the corners (in the same order).

In [ ]:
# Real world coordinates in (x, y, z).
locations = [
    (0, 90, 36),
    (0, 72, 18),
    (0, 90, 72),
    (54, 90, 0),
    (36, 108, 0),
    (18, 36, 0),
]

(2 points) Now that we have the real world coordinates and the pixel coordinates, we can find $p$, the projection matrix. To do this, we can set up the equation $Ap=0$ in the slides. Set up $A$ below. Refer to the slides in the lecture on how to construct $A$.

A trivial solution is when the projection matrix is all zeros, but that obviously isn't useful for us. Additionally, since our input data may be noisy, there may not be a perfect $p$ that exactly satisfies $Ap=0$. Hence we want to find the least squares solution. As shown in the appendix of the lecture slides, the eigenvector $p$ with the smallest eigenvalue $\lambda$ of matrix $A^T A$ is the solution to our problem.

(2 points) In the cell below, find the eigenvectors of $A^T A$, extract the eigenvector corresponding to the smallest eigenvalue, and reshape it into a 3x4 matrix $P$. Print the result. Since $A^T A$ is real and symmetric, you may want to use [`np.linalg.eigh()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html) instead of `np.linalg.eig()` for faster execution.

Alternatively, you can choose to perform singular value decomposition on $A$ and recover $P$ from the SVD results.

In [ ]:
# TODO

np.set_printoptions(precision=5, suppress=True)
print(P)

(1 point) Now that we have the projection matrix $P$, we can verify that it can map the 3D world coordinates to 2D image coordinates. In the cell below, print the projection results using $P$ along side the original 2D corner coordinates to verify that they are close. 

(1 point) Given the projection matrix $P$, we can recover the extrinsic rotation $R$ and the intrinsic matrix $K$ via RQ Decomposition. To do so, first normalize $P$ so the L2 norm of `P[2, :3]` is 1. Print the result.

(1 point) Using `scipy.linalg.rq()`, decompose `P[:3, :3]` into an upper triangular matrix $K$ and an orthonormal matrix $R$. Print both matrices.

In [ ]:
import scipy.linalg
K, R = 
print(K)
print(R)

You may notice that the intrinsic matrix $K$ has negative values on its diagonal, which does not seem right since the focus length of our camera should be a positive value. This is because `scipy.linalg.rq()` does not force positive diagonal values. We can do so manually by manipulating the matrices a little bit.

In [ ]:
S = np.diag(np.sign(np.diag(K)))
K = K @ S
R = S @ R
print(K)
print(R)

(1 point) Using $K$ we can then recover the extrinsic translation $t$. Do so below, and print the results.